In [1]:
config = {
    "pretrained_model": "roberta-base",
    "tokenizer": "roberta-base",
    "max_seq_length": 512,
    "batch_size": 10,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "0,1,2,3",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/bert_cased_512_biocaster_4cate_20200903/",
    "patience": 4,
}

In [2]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaster2df
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaster2df(data_file)
data_df["source"] = "Biocaster"
data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,3,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,3,Biocaster


In [3]:
from sklearn.utils import shuffle
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
data_df.iloc[:train_index]["flag"]="train"
data_df.iloc[train_index:valid_index]["flag"]="valid"
data_df.iloc[valid_index:]["flag"]="test"

<ipython-input-3-79406d09754d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[:train_index]["flag"]="train"
<ipython-input-3-79406d09754d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-3-79406d09754d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [4]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [5]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [6]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at roberta-base were not used when initializing BertForSequenceClassification: ['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediate.dense.bias', 'rober

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50265, 768)
bert.embeddings.position_embeddings.weight                (514, 768)
bert.embeddings.token_type_embeddings.weight                (1, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [8]:
cls.train(train_set,dev_set)

/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '



Execute [train_an_epoch] method costing 27726.69 ms
[Epoch 0] loss: 83.46884447336197



Execute [eval] method costing 1074.89 ms
--------------------------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.67     |
|  1 | macro_precision | 0.1675   |
|  2 | macro_recall    | 0.25     |
|  3 | macro_f1        | 0.200599 |
|  4 | micro_precision | 0.67     |
|  5 | micro_recall    | 0.67     |
|  6 | micro_f1        | 0.67     |
+----+-----------------+----------+
--------------------------------------------------------------------------------


/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Execute [train_an_epoch] method costing 21877.56 ms
[Epoch 1] loss: 76.66093903779984


...


Execute [eval] method costing 1102.95 ms
--------------------------------------------------------------------------------
[Epoch 7]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.77     |
|  1 | macro_precision | 0.355495 |
|  2 | macro_recall    | 0.415801 |
|  3 | macro_f1        | 0.378657 |
|  4 | micro_precision | 0.77     |
|  5 | micro_recall    | 0.77     |
|  6 | micro_f1        | 0.77     |
+----+-----------------+----------+
--------------------------------------------------------------------------------



Execute [train_an_epoch] method costing 22440.33 ms
[Epoch 8] loss: 46.8002030197531



Execute [eval] method costing 1069.28 ms



Execute [train_an_epoch] method costing 22514.42 ms
[Epoch 9] loss: 47.166070241481066



Execute [eval] method costing 1052.90 ms



In [9]:
cls.test(test_set)


Execute [eval] method costing 1096.47 ms


{'accuracy': 0.7623762376237624,
 'macro_precision': 0.39338235294117646,
 'macro_recall': 0.3667834051724138,
 'macro_f1': 0.3650117508813161,
 'micro_precision': 0.7623762376237624,
 'micro_recall': 0.7623762376237624,
 'micro_f1': 0.7623762376237624}

In [10]:
test_set.iloc[1:10]

,index,docs,labels,source,flag
903,303,\n Thong surges to first national title \n14:...,0,Biocaster,test
904,699,\n Cat death prompts bird flu advice \n \n \n...,3,Biocaster,test
905,959,\n\nDeath toll in Mrashtra bus accident rises ...,0,Biocaster,test
906,31,\nDrug makers talk merger \nBig News Network....,0,Biocaster,test
907,260,\n Italians prepare for weekend elections\nAp...,0,Biocaster,test
908,237,\nA clearer picture on beach pollution\n \n Ap...,0,Biocaster,test
909,279,\n Viet nam to boost mining of lead and zinc ...,0,Biocaster,test
910,520,"\n FOOT and MOUTH DISEASE, BOVINE - CHINA : (...",3,Biocaster,test
911,780,\n West Nile Found In Blood Donation\nWASHING...,3,Biocaster,test


In [11]:
cls.predict(test_set.iloc[1:10])


Execute [predict] method costing 160.04 ms


array([0, 0, 0, 0, 0, 0, 0, 0, 0])